In [1]:
from lionagi import Branch, iModel

In [2]:
from pydantic import BaseModel

llama33 = iModel(
    provider="groq",
    model="llama-3.3-70b-versatile",
)

gpt4o = iModel(
    provider="openai",
    model="gpt-4o-mini",
)


class Test(BaseModel):
    """
    Test model
    """

    comments: str


async def test_func(query: str):
    """
    this is a test function it will return the query as it is

    Args:
        query: str: query to be returned

    Returns:
        str: query as it is
    """
    return query


french = Branch(
    imodel=llama33,
    tools=test_func,
)

In [3]:
result = await french.operate(
    instruction="hello? you should test the test_func with a sample queries",
    operative_model=Test,
    actions=True,
)

In [4]:
result

Test(comments='Testing the test_func with a sample query')

In [5]:
french.imodel.sequential_exchange

True

In [6]:
french.to_df()

,timestamp,lion_class,role,content,id,sender,recipient,metadata
0,2024-12-25 05:06:07.973422+00:00,Instruction,user,{'context': [{'respond_schema_info': {'$defs':...,8a65911c-a909-49b0-8bc8-10baf041751a,user,14132072-3812-4b58-ac53-065b37a19b75,{'last_updated': {'context': 1735103168.417341}}
1,2024-12-25 05:06:08.417155+00:00,AssistantResponse,assistant,"{'assistant_response': '```json { ""comments""...",ddb5279b-81d3-498b-ae9e-bb45eedbce8d,14132072-3812-4b58-ac53-065b37a19b75,N/A,"{'model_response': {'choices': [{'index': 0, '..."
2,2024-12-25 05:06:08.418792+00:00,ActionRequest,assistant,"{'action_request': {'function': 'test_func', '...",9f2601af-88f2-4f69-8f69-44753469d3c0,14132072-3812-4b58-ac53-065b37a19b75,0f7df12b-e07c-4c01-a52f-9a892ce0564d,{}
3,2024-12-25 05:06:08.418938+00:00,ActionResponse,assistant,{'action_request_id': 9f2601af-88f2-4f69-8f69-...,cff04174-d7df-4c8d-a66a-8a3fc877ba2b,0f7df12b-e07c-4c01-a52f-9a892ce0564d,14132072-3812-4b58-ac53-065b37a19b75,{}


In [7]:
french.msgs.last_response.model_response

{'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '```json\n{\n  "comments": "Testing the test_func with a sample query",\n  "action_requests": [\n    {\n      "function": "test_func",\n      "arguments": {\n        "query": "This is a sample query"\n      }\n    }\n  ],\n  "action_required": true\n}\n```'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'model': 'llama-3.3-70b-versatile',
 'usage': {'queue_time': 0.000735520999999989,
  'prompt_tokens': 608,
  'prompt_time': 0.064599073,
  'completion_tokens': 66,
  'completion_time': 0.24,
  'total_tokens': 674,
  'total_time': 0.304599073}}

In [8]:
a = french.msgs.instructions[0]

In [10]:
a

Instruction(id=IDType(8a65911c-a909-49b0-8bc8-10baf041751a), created_timestamp=2024-12-25 05:06:07.973422, content={'context': [{'respond_schema_info': {'$defs': {'ActionRequestModel': {'properties': {'function':..., metadata={'last_updated': {'context': 1735103168.417341}}, created_timestamp=1735103167.973422, sender=user, recipient=14132072-3812-4b58-ac53-065b37a19b75, role=user, extra_fields={})

In [11]:
french.msgs.instructions[0].tool_schemas

{'tools': [{'type': 'function',
   'function': {'name': 'test_func',
    'description': 'this is a test function it will return the query as it is',
    'parameters': {'type': 'object',
     'properties': {'query': {'type': 'string',
       'description': 'str: query to be returned'}},
     'required': ['query']}}}]}

In [12]:
from IPython.display import Markdown

Markdown(french.msgs.instructions[0].chat_msg["content"])


---
 # Task
## **Task context**
- respond_schema_info: {'$defs': {'ActionRequestModel': {'properties': {'function': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': "Name of the function to call from the provided `tool_schemas`. If no `tool_schemas` exist, set to None or leave blank. Never invent new function names outside what's given.", 'examples': ['add', 'multiply', 'divide'], 'title': 'Function'}, 'arguments': {'anyOf': [{'type': 'object'}, {'type': 'null'}], 'description': 'Dictionary of arguments for the chosen function. Use only argument names/types defined in `tool_schemas`. Never introduce extra argument names.', 'examples': [{'num1': 1, 'num2': 2}, {'x': 'hello', 'y': 'world'}], 'title': 'Action Arguments'}}, 'title': 'ActionRequestModel', 'type': 'object'}}, 'properties': {'comments': {'title': 'Comments', 'type': 'string'}, 'action_requests': {'description': 'List of actions to be executed when `action_required` is true. Each action must align with the available `tool_schemas`. Leave empty if no actions are needed.', 'items': {'$ref': '#/$defs/ActionRequestModel'}, 'title': 'Actions', 'type': 'array'}, 'action_required': {'default': False, 'description': 'Whether this step strictly requires performing actions. If true, the requests in `action_requests` must be fulfilled, assuming `tool_schemas` are available. If false or no `tool_schemas` exist, actions are optional.', 'title': 'Action Required', 'type': 'boolean'}}, 'required': ['comments'], 'title': 'Test', 'type': 'object'} 



## **Task instruction**
hello? you should test the test_func with a sample queries


## **Task tool_schemas**
- tools: [{'type': 'function', 'function': {'name': 'test_func', 'description': 'this is a test function it will return the query as it is', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'str: query to be returned'}}, 'required': ['query']}}}] 



## **Task response format**
**MUST RETURN JSON-PARSEABLE RESPONSE ENCLOSED BY JSON CODE BLOCKS.** 
```json
{'comments': <class 'str'>, 'action_requests': [{'function': str | None, 'arguments': dict[str, typing.Any] | None}], 'action_required': <class 'bool'>}
```




---
